In [1]:

from goal_model_step.model import StepGoalModel
import torch


2023-10-13 17:44:14.208562: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-13 17:44:14.228747: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-13 17:44:14.524794: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[2023-10-13 17:44:18,414] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
goal_path = 'goal_step.ckpt'

In [3]:

goal_model = StepGoalModel.load(goal_path, device=torch.device("cpu"), freeze=True)



Processing zero checkpoint 'goal.ckpt/checkpoint'
Detected checkpoint of type zero stage 2, world_size: 1
Parsing checkpoint created by deepspeed==0.10.3
Reconstructed fp32 state dict with 170 params 299637760 elements
Saving fp32 state dict to /tmp/tmphf7pz5qb/lightning.cpkt


In [37]:
out = goal_model.batch_generate(['<extra_id_0>asdfasdf', 'sdfdf', 'asdfasdfasdf',"<extra_id_0>n : ℕ\n⊢ gcd n n = n" ])

tensor([[    -inf, -22.1560,     -inf,  ...,     -inf,     -inf,     -inf],
        [    -inf, -18.0547,     -inf,  ...,     -inf,     -inf,     -inf],
        [    -inf, -18.6227,     -inf,  ...,     -inf,     -inf,     -inf],
        [    -inf, -18.4882,     -inf,  ...,     -inf,     -inf,     -inf]])
torch.Size([4, 384])


In [39]:
out

tensor([-83.7897, -70.1934, -63.5459, -60.6499])

In [38]:
torch.log_softmax(out, 0)

tensor([-23.1936,  -9.5973,  -2.9498,  -0.0538])

In [41]:
import pickle
with open("traces/failed_convex.is_linear_preimage_2023-10-13_20:12.pk", "rb") as f:
    trace = pickle.load(f)

In [48]:
trace.tree.state.goals[0]

Goal(assumptions=[Declaration(ident='𝕜', lean_type='Type u_1'), Declaration(ident='E', lean_type='Type u_2'), Declaration(ident='F', lean_type='Type u_3'), Declaration(ident='_inst_1', lean_type='ordered_semiring 𝕜'), Declaration(ident='_inst_2', lean_type='add_comm_monoid E'), Declaration(ident='_inst_3', lean_type='add_comm_monoid F'), Declaration(ident='_inst_4', lean_type='module 𝕜 E'), Declaration(ident='_inst_5', lean_type='module 𝕜 F'), Declaration(ident='s', lean_type='set F'), Declaration(ident='hs', lean_type='convex 𝕜 s'), Declaration(ident='f', lean_type='E → F'), Declaration(ident='hf', lean_type='is_linear_map 𝕜 f')], conclusion="convex 𝕜 (f ⁻¹' s)")

In [51]:
nodes = trace.nodes

In [90]:
outs = [o.tactic for o in trace.tree.out_edges]


In [91]:
outs

['intros x hx y hy a b ha hb hab',
 'rintro x hx y hy a b ha hb hab',
 'classical',
 'intros x hx',
 'intros x hx y hy a b ha hb hab ab',
 'intros a ha b hb hab',
 'intros p hp x hx y hy a b ha hb hab',
 'intros y hy a b ha hb hab',
 'intros x₁ hx₁ x₂ hx₂ a b ha hb hab',
 'intros x hx y hy hxy a b ha hb hab',
 'refine λ x hx y hy a b ha hb hab, _',
 'intros a ha x hx y hy a b ha hb hab',
 'rintros x hx y hy a b ha hb hab',
 'rintro _ ⟨x, hx, rfl⟩ _ ⟨y, hy, rfl⟩ hxy a b ha hb hab',
 'intros x₁ hx₁ x₂ hx y hy a b ha hb hab',
 'intros x hx y hy _ a b ha hb hab',
 'intros r hr x hx y hy a b ha hb hab',
 'intros a b ha hb hab',
 'intros a ha b hb ab',
 'intros x hx y hy h',
 'intros e he x hx y hy a b ha hb hab',
 'obtain rfl | h := s.eq_empty_or_nonempty',
 'intros e x hx y hy a b ha hb hab',
 'intros a ha x hx a b ha hb hab',
 'intros x hx y hy a b ha hb hab hab',
 'intros x₁ hx₁ x₂ h',
 'rintro a ha x hx y hy a b ha hb hab',
 'intros a b ha hb ab',
 'intros x₁ hx₁ x₂ hx hy a b ha hb hab'

In [101]:
child_goals = [nodes[child].state.goals[0] for child in trace.tree.children if len(nodes[child].state.goals) == 1]

In [104]:
[a.ident for a in child_goals[3].assumptions]

['𝕜',
 'E',
 'F',
 '_inst_1',
 '_inst_2',
 '_inst_3',
 '_inst_4',
 '_inst_5',
 's',
 'hs',
 'f',
 'hf',
 'r',
 'hr',
 'a',
 'b',
 'ha',
 'hb',
 'hab']

In [106]:
child_goals[3].conclusion.split(' ')

['0',
 '≤',
 'hb',
 '→',
 'ha',
 '+',
 'hb',
 '=',
 '1',
 '→',
 'ha',
 '•',
 'r',
 '+',
 'hb',
 '•',
 'a',
 '∈',
 'f',
 "⁻¹'",
 's']

In [ ]:
split_chars  = ['⦃', '⦄', '[', ']', '(', ')', '-']

In [ ]:
import re
_TOKEN_RE = re.compile(r'⦃|⦄|,|\{|}|\(|\)|\[|\]|[^\s@⊢:(){}[]]+')


In [107]:
decs = [(a.ident, a.lean_type.split(' ')) for c in child_goals for a in c.assumptions]


In [108]:
decs

[('𝕜', ['Type', 'u_1']),
 ('E', ['Type', 'u_2']),
 ('F', ['Type', 'u_3']),
 ('_inst_1', ['ordered_semiring', '𝕜']),
 ('_inst_2', ['add_comm_monoid', 'E']),
 ('_inst_3', ['add_comm_monoid', 'F']),
 ('_inst_4', ['module', '𝕜', 'E']),
 ('_inst_5', ['module', '𝕜', 'F']),
 ('s', ['set', 'F']),
 ('hs', ['convex', '𝕜', 's']),
 ('f', ['E', '→', 'F']),
 ('hf', ['is_linear_map', '𝕜', 'f']),
 ('y', ['E']),
 ('hy', ['y', '∈', 'f', "⁻¹'", 's']),
 ('a', ['E']),
 ('b', ['a', '∈', 'f', "⁻¹'", 's']),
 ('ha', ['𝕜']),
 ('hb', ['𝕜']),
 ('hab', ['0', '≤', 'ha']),
 ('𝕜', ['Type', 'u_1']),
 ('E', ['Type', 'u_2']),
 ('F', ['Type', 'u_3']),
 ('_inst_1', ['ordered_semiring', '𝕜']),
 ('_inst_2', ['add_comm_monoid', 'E']),
 ('_inst_3', ['add_comm_monoid', 'F']),
 ('_inst_4', ['module', '𝕜', 'E']),
 ('_inst_5', ['module', '𝕜', 'F']),
 ('s', ['set', 'F']),
 ('hs', ['convex', '𝕜', 's']),
 ('f', ['E', '→', 'F']),
 ('hf', ['is_linear_map', '𝕜', 'f']),
 ('x', ['E']),
 ('hx', ['x', '∈', 'f', "⁻¹'", 's']),
 ('y', ['E']),

In [64]:
child_goals[0]

"0 ≤ hb → ha + hb = 1 → ha • y + hb • a ∈ f ⁻¹' s"

In [65]:
child_goals[2]

"a • x₁ + b • x₂ ∈ f ⁻¹' s"

In [3]:
prev_goals= ['ι : Type u_1,\n𝕜 : Type u_2,\n_inst_1 : is_R_or_C 𝕜,\nG : ι → Type u_4,\n_inst_4 : Π (i : ι), normed_add_comm_group (G i),\n_inst_5 : Π (i : ι), inner_product_space 𝕜 (G i),\nf g : ↥(lp G 2),\nthis : true\n⊢ ∀ (i : ι), inner (⇑f i) (⇑g i) = inner (⇑f i) (⇑g i)', 'ι : Type u_1,\n𝕜 : Type u_2,\n_inst_1 : is_R_or_C 𝕜,\nG : ι → Type u_4,\n_inst_4 : Π (i : ι), normed_add_comm_group (G i),\n_inst_5 : Π (i : ι), inner_product_space 𝕜 (G i),\nf g : ↥(lp G 2),\nthis : true,\nthis : ∀ (i : ι), inner (⇑f i) (⇑g i) = inner (⇑f i) (⇑g i)\n⊢ has_sum (λ (i : ι), inner (⇑f i) (⇑g i)) 2', 'ι : Type u_1,\n𝕜 : Type u_2,\n_inst_1 : is_R_or_C 𝕜,\nG : ι → Type u_4,\n_inst_4 : Π (i : ι), normed_add_comm_group (G i),\n_inst_5 : Π (i : ι), inner_product_space 𝕜 (G i),\nf g : ↥(lp G 2)\n⊢ ∀ (i : ι), inner (⇑f i) (⇑g i) = inner (⇑f i) (⇑g i)']

In [4]:
len(prev_goals)

3

In [6]:

response = ['ι : Type u_1,\n𝕜 : Type u_2,\n_inst_1 : is_R_or_C 𝕜,\nG : ι → Type u_4,\n_inst_4 : Π (i : ι), normed_add_comm_group (G i),\n_inst_5 : Π (i : ι), inner_product_space 𝕜 (G i),\nf g : ↥(lp G 2)\n⊢ ∀ (i : ι), inner (⇑f i) (⇑g i) = inner (⇑f i) (⇑g i)',
'ι : Type u_1,\n𝕜 : Type u_2,\n_inst_1 : is_R_or_C 𝕜,\nG : ι → Type u_4,\n_inst_4 : Π (i : ι), normed_add_comm_group (G i),\n_inst_5 : Π (i : ι), inner_product_space 𝕜 (G i),\nf g : ↥(lp G 2),\nthis : true,\nthis : ∀ (i : ι), inner (⇑f i) (⇑g i) = inner (⇑f i) (⇑g i)\n⊢ has_sum (λ (i : ι), inner (⇑f i) (⇑g i)) 2',
'ι : Type u_1,\n𝕜 : Type u_2,\n_inst_1 : is_R_or_C 𝕜,\nG : ι → Type u_4,\n_inst_4 : Π (i : ι), normed_add_comm_group (G i),\n_inst_5 : Π (i : ι), inner_product_space 𝕜 (G i),\nf g : ↥(lp G 2)\n⊢ ∀ (i : ι), inner (⇑f i) (⇑g i) = inner (⇑f i) (⇑g i)']

tac_state = ['ι : Type u_1,\n𝕜 : Type u_2,\n_inst_1 : is_R_or_C 𝕜,\nG : ι → Type u_4,\n_inst_4 : Π (i : ι), normed_add_comm_group (G i),\n_inst_5 : Π (i : ι), inner_product_space 𝕜 (G i),\nf g : ↥(lp G 2),\nthis : true\n⊢ ∀ (i : ι), inner (⇑f i) (⇑g i) = inner (⇑f i) (⇑g i)',
'ι : Type u_1,\n𝕜 : Type u_2,\n_inst_1 : is_R_or_C 𝕜,\nG : ι → Type u_4,\n_inst_4 : Π (i : ι), normed_add_comm_group (G i),\n_inst_5 : Π (i : ι), inner_product_space 𝕜 (G i),\nf g : ↥(lp G 2),\nthis : true,\nthis : ∀ (i : ι), inner (⇑f i) (⇑g i) = inner (⇑f i) (⇑g i)\n⊢ has_sum (λ (i : ι), inner (⇑f i) (⇑g i)) 2',
'ι : Type u_1,\n𝕜 : Type u_2,\n_inst_1 : is_R_or_C 𝕜,\nG : ι → Type u_4,\n_inst_4 : Π (i : ι), normed_add_comm_group (G i),\n_inst_5 : Π (i : ι), inner_product_space 𝕜 (G i),\nf g : ↥(lp G 2)\n⊢ ∀ (i : ι), inner (⇑f i) (⇑g i) = inner (⇑f i) (⇑g i)']


In [39]:
[g for g in response if g not in prev_goals]

[]

In [48]:
response[1] == prev_goals[1]

True

In [43]:
response[0]

'ι : Type u_1,\n𝕜 : Type u_2,\n_inst_1 : is_R_or_C 𝕜,\nG : ι → Type u_4,\n_inst_4 : Π (i : ι), normed_add_comm_group (G i),\n_inst_5 : Π (i : ι), inner_product_space 𝕜 (G i),\nf g : ↥(lp G 2)\n⊢ ∀ (i : ι), inner (⇑f i) (⇑g i) = inner (⇑f i) (⇑g i)'

In [45]:
prev_goals[2]

'ι : Type u_1,\n𝕜 : Type u_2,\n_inst_1 : is_R_or_C 𝕜,\nG : ι → Type u_4,\n_inst_4 : Π (i : ι), normed_add_comm_group (G i),\n_inst_5 : Π (i : ι), inner_product_space 𝕜 (G i),\nf g : ↥(lp G 2)\n⊢ ∀ (i : ι), inner (⇑f i) (⇑g i) = inner (⇑f i) (⇑g i)'

In [53]:
response[2]

'ι : Type u_1,\n𝕜 : Type u_2,\n_inst_1 : is_R_or_C 𝕜,\nG : ι → Type u_4,\n_inst_4 : Π (i : ι), normed_add_comm_group (G i),\n_inst_5 : Π (i : ι), inner_product_space 𝕜 (G i),\nf g : ↥(lp G 2)\n⊢ ∀ (i : ι), inner (⇑f i) (⇑g i) = inner (⇑f i) (⇑g i)'

In [46]:
response[1]

'ι : Type u_1,\n𝕜 : Type u_2,\n_inst_1 : is_R_or_C 𝕜,\nG : ι → Type u_4,\n_inst_4 : Π (i : ι), normed_add_comm_group (G i),\n_inst_5 : Π (i : ι), inner_product_space 𝕜 (G i),\nf g : ↥(lp G 2),\nthis : true,\nthis : ∀ (i : ι), inner (⇑f i) (⇑g i) = inner (⇑f i) (⇑g i)\n⊢ has_sum (λ (i : ι), inner (⇑f i) (⇑g i)) 2'

In [47]:
prev_goals[1]

'ι : Type u_1,\n𝕜 : Type u_2,\n_inst_1 : is_R_or_C 𝕜,\nG : ι → Type u_4,\n_inst_4 : Π (i : ι), normed_add_comm_group (G i),\n_inst_5 : Π (i : ι), inner_product_space 𝕜 (G i),\nf g : ↥(lp G 2),\nthis : true,\nthis : ∀ (i : ι), inner (⇑f i) (⇑g i) = inner (⇑f i) (⇑g i)\n⊢ has_sum (λ (i : ι), inner (⇑f i) (⇑g i)) 2'

In [49]:
prev_goals[0]

'ι : Type u_1,\n𝕜 : Type u_2,\n_inst_1 : is_R_or_C 𝕜,\nG : ι → Type u_4,\n_inst_4 : Π (i : ι), normed_add_comm_group (G i),\n_inst_5 : Π (i : ι), inner_product_space 𝕜 (G i),\nf g : ↥(lp G 2),\nthis : true\n⊢ ∀ (i : ι), inner (⇑f i) (⇑g i) = inner (⇑f i) (⇑g i)'

In [51]:
response[2] in prev_goals

True

In [52]:
prev_goals.index(response[2])

2

In [9]:
len(tac_state)

3

In [26]:
tac_state[0] == response[2]

False

In [17]:
response[2] in prev_goals

True

In [38]:
response[2]

'ι : Type u_1,\n𝕜 : Type u_2,\n_inst_1 : is_R_or_C 𝕜,\nG : ι → Type u_4,\n_inst_4 : Π (i : ι), normed_add_comm_group (G i),\n_inst_5 : Π (i : ι), inner_product_space 𝕜 (G i),\nf g : ↥(lp G 2)\n⊢ ∀ (i : ι), inner (⇑f i) (⇑g i) = inner (⇑f i) (⇑g i)'

In [37]:
tac_state[2]

'ι : Type u_1,\n𝕜 : Type u_2,\n_inst_1 : is_R_or_C 𝕜,\nG : ι → Type u_4,\n_inst_4 : Π (i : ι), normed_add_comm_group (G i),\n_inst_5 : Π (i : ι), inner_product_space 𝕜 (G i),\nf g : ↥(lp G 2)\n⊢ ∀ (i : ι), inner (⇑f i) (⇑g i) = inner (⇑f i) (⇑g i)'